# KFold Cross Validation.

In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.datasets import load_digits
import matplotlib.pyplot as plt
digits = load_digits()

In [2]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(digits.data,digits.target,test_size=0.3)

#### samples we put in these 4 sets X_train, X_test, y_train, y_test changes hence score also changes

## Logistic Regression.

In [3]:
lr = LogisticRegression(solver='liblinear',multi_class='ovr') #classifier
lr.fit(X_train, y_train)
lr.score(X_test, y_test)

0.95

## SVM (Support Vector Machine)

In [4]:
svm = SVC(gamma='auto')
svm.fit(X_train, y_train)
svm.score(X_test, y_test)

0.44074074074074077

## Random Forest

In [5]:
rf = RandomForestClassifier(n_estimators=40)
rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.9592592592592593

# KFold cross validation

## Example 
## step 1 initialization

In [6]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=3) #3 folds
kf

KFold(n_splits=3, random_state=None, shuffle=False)

#### example
#### 1 fold is for testing and remaing 2 folds are for training


In [7]:
for train_index, test_index in kf.split([1,2,3,4,5,6,7,8,9]):
    print(train_index, test_index)

[3 4 5 6 7 8] [0 1 2]
[0 1 2 6 7 8] [3 4 5]
[0 1 2 3 4 5] [6 7 8]


## step 2 Use KFold for our digits example

In [8]:
def get_score(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    return model.score(X_test, y_test)

In [23]:
from sklearn.model_selection import StratifiedKFold #separation of categories occur in a uniform way
folds = StratifiedKFold(n_splits=3) #3 folds
#Initializing Lists to Store evaluation Scores
scores_logistic = []
scores_svm = []
scores_rf = []

for train_index, test_index in folds.split(digits.data,digits.target): 
    X_train, X_test, y_train, y_test = digits.data[train_index], digits.data[test_index], \
                                       digits.target[train_index], digits.target[test_index]
    scores_logistic.append(get_score(LogisticRegression(solver='liblinear',multi_class='ovr'), X_train, X_test, y_train, y_test))  
    scores_svm.append(get_score(SVC(gamma='auto'), X_train, X_test, y_train, y_test))
    scores_rf.append(get_score(RandomForestClassifier(n_estimators=40), X_train, X_test, y_train, y_test))

In [24]:
scores_logistic

[0.8948247078464107, 0.9532554257095158, 0.9098497495826378]

In [25]:
scores_svm

[0.3806343906510851, 0.41068447412353926, 0.5125208681135225]

In [26]:
scores_rf

[0.9248747913188647, 0.9465776293823038, 0.9248747913188647]

# cross_val_score function

In [27]:
from sklearn.model_selection import cross_val_score

## Logistic regression model performance using cross_val_score

In [28]:
cross_val_score(LogisticRegression(solver='liblinear',multi_class='ovr'), digits.data, digits.target, cv=3) #folds

array([0.89482471, 0.95325543, 0.90984975])

#### solver='liblinear': This is the solver used for the optimization problem in logistic regression.multi_class='ovr': This specifies that the logistic regression should use the "one-vs-rest" strategy for multi-class classification.

## svm model performance using cross_val_score

In [29]:
cross_val_score(SVC(gamma='auto'), digits.data, digits.target,cv=3)


array([0.38063439, 0.41068447, 0.51252087])

##### This sets the kernel coefficient for the 'rbf' kernel. 'auto' means that the value of gamma will be set to 1/n_features, where n_features is the number of features in the dataset.

## random forest performance using cross_val_score

In [30]:
cross_val_score(RandomForestClassifier(n_estimators=40),digits.data, digits.target,cv=3)

array([0.92320534, 0.95158598, 0.93155259])

### cross_val_score uses stratifield kfold by default

# Parameter tunning using k fold cross validation

In [31]:
scores1 = cross_val_score(RandomForestClassifier(n_estimators=5),digits.data, digits.target, cv=10)
np.average(scores1)

0.8748075729360645

In [32]:
scores2 = cross_val_score(RandomForestClassifier(n_estimators=20),digits.data, digits.target, cv=10)
np.average(scores2)

0.9404593420235876

In [33]:
scores3 = cross_val_score(RandomForestClassifier(n_estimators=30),digits.data, digits.target, cv=10)
np.average(scores3)

0.9393451272501551

In [34]:
scores4 = cross_val_score(RandomForestClassifier(n_estimators=40),digits.data, digits.target, cv=10)
np.average(scores4)

0.9410055865921787

### Here we used cross_val_score to fine tune our random forest classifier and figured that having around 40 trees in random forest gives best result.